In [ ]:
import requests # pip install requests
import json
import os
import time
from tqdm.notebook import tqdm # pip install tqdm

This notebook will save your slack

In [ ]:
# use OAuth user token instead of emodji. create an app with a token here https://api.slack.com/apps/

In [ ]:
token = ':)'

In [ ]:
authorization=f"Bearer {token}"

# how to get last messages from a channel

In [ ]:
channel = '<some_channel>' # подставить свой
url = f"https://slack.com/api/conversations.history?channel={channel}" 
r = requests.post(url, headers={'Content-type': 'application/json', "Authorization": authorization})

In [ ]:
r.json()

# How to download all direct messages

## map users' ids to conversations' ids

In [ ]:
url = "https://slack.com/api/conversations.list?types=im"
r = requests.post(url, headers={'Content-type': 'application/json', "Authorization": authorization})

In [ ]:
user_id_to_dm = {}
for ch in r.json()['channels']:
    user_id_to_dm[ch['user']] = ch['id']
print(user_id_to_dm)

In [ ]:
len(user_id_to_dm)

## map users' ids to their names

In [ ]:
url = "https://slack.com/api/users.list"
r = requests.post(url, headers={'Content-type': 'application/json',
                                "Authorization": authorization})

In [ ]:
user_id_to_name = {}
for member in r.json()['members']:
    user_id_to_name[member['id']] = member['profile']['real_name']
user_id_to_name

In [ ]:
len(user_id_to_name)

In [ ]:
result_dir = 'slack-history'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

## download direct messages and threads (пока без файлов, но всё будет!)

In [ ]:
def load_messages(url, authorization, dir_name, messages, counter):
    r = requests.post(url, headers={'Content-type': 'application/json',
                                    "Authorization": authorization,
                                   })
    data = r.json()
    number_of_messages = len(data['messages'])
    print(number_of_messages)
    
    response_metadata = data.get('response_metadata')
    for message in data['messages']:
        msg_timestamp = message['ts']
        url =  f"https://slack.com/api/conversations.replies?channel={dm_id}&ts={msg_timestamp}&limit=1000"
        r = requests.post(url, headers={'Content-type': 'application/json',
                                    "Authorization": authorization,
                                   })
        messages.append(r.json())
        time.sleep(0.05)
    return response_metadata, counter, messages

In [ ]:
for i, (user_id, dm_id) in enumerate(tqdm(user_id_to_dm.items())):
    name = user_id_to_name.get(user_id, user_id)
    print(name)
    # name it so that every user would be unique
    # there can be many users with the same real names
    filename = os.path.join(result_dir, name + f"_{i}.json")

    url = f"https://slack.com/api/conversations.history?channel={dm_id}&limit=200"
    messages = []
    response_metadata, counter, messages = load_messages(url,
                                                         authorization,
                                                         dir_name,
                                                         messages,
                                                         counter=0
                                                        )
    if response_metadata is not None:
        next_cursor = response_metadata['next_cursor']
        while next_cursor:
            counter += 1
            url = f"https://slack.com/api/conversations.history?channel={dm_id}&limit=200&cursor={next_cursor}"
            response_metadata, counter, messages = load_messages(url,
                                                                authorization,
                                                                dir_name,
                                                                messages,
                                                                counter
                                                               )
            if response_metadata is not None:
                next_cursor = response_metadata['next_cursor'].replace("=", "%3D")
            else:
                next_cursor = ''
            
            
    with open(filename, 'w') as f:
        json.dump({'messages':messages}, f)